In [15]:
import pandas as pd
import numpy as np
import os
import sys

df = pd.read_csv('ore_italy.csv', delimiter = '\t',encoding='utf-8')
#print(df)

sample_id_counter = 0
analysis_id = -1
analysis_counter = -1
location_id = -1
location_counter = -1
site_name = []
site_name_with_id = []
analysis_group = []
analysis_group_with_id = []
#df = df.head(100)

f=open('insert.sql',"w+",encoding='utf8')
df = df.reset_index()  # make sure indexes pair with number of rows

for index, row in df.iterrows():
    if row['site_name'] not in site_name :
        site_name.append(row['site_name'])
        location_counter +=1
        location_id = location_counter
        site_name_with_id.append(location_counter)
        site_name_with_id.append(row['site_name'])
        f.write('INSERT INTO public.mctor_locations (id_location,level_value)\nVALUES (\''+str(location_id)+'\',\''+row['site_name']+'\');\n')
        f.write('INSERT INTO service.mctor_loc_political (town,region,country,continent)\nVALUES (\''+row['town']+'\',\''+row['region_province']+'\',\''+row['country_state']+'\',\'Europe\');\n')

    else :   
        for k in range(1,len(site_name_with_id)) :
            if row['site_name'] == site_name_with_id[k] :
                location_id = site_name_with_id[k-1]
    
    if row['collected_by'] not in analysis_group :
        analysis_group.append(row['collected_by'])
        analysis_counter +=1
        analysis_id = analysis_counter
        analysis_group_with_id.append(analysis_counter)
        analysis_group_with_id.append(row['collected_by'])
        f.write('INSERT INTO public.mctor_an_meta (id_analysis, team)\nVALUES (\''+str(analysis_id)+'\',\''+row['collected_by']+'\');\n')
    else :   
        for k in range(1,len(analysis_group_with_id)) :
            if row['collected_by'] == analysis_group_with_id[k] :
                analysis_id = analysis_group_with_id[k-1]
                
    f.write('INSERT INTO public.mctor_samples (id_sample, id_location, type)\nVALUES (\''+str(sample_id_counter)+'\',\''+str(location_id)+'\',\'O\');\n')
    f.write('INSERT INTO public.mctor_an_pb_isotopes (id_sample,id_analysis,pb_iso_208_206,pb_iso_207_206,pb_iso_206_204)\nVALUES (\''+str(sample_id_counter)+'\',\''+str(analysis_id)+'\',\''+str(round(row['208Pb/206Pb'],7))+'\',\''+str(round(row['207Pb/206Pb'],7))+'\',\''+str(round(row['206Pb/204Pb'],7))+'\');\n')
    sample_id_counter+=1

f.close()